## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import cv2
import math
import numpy as np

sys.path.append('../pyslam/')
from config import Config
from visual_odometry import VisualOdometry
from camera  import PinholeCamera
from ground_truth import groundtruth_factory
from dataset import dataset_factory
import matplotlib.pyplot as plt
from glob import glob

from feature_tracker import feature_tracker_factory, FeatureTrackerTypes 
from feature_manager import feature_manager_factory
from feature_types import FeatureDetectorTypes, FeatureDescriptorTypes, FeatureInfo
from feature_matcher import feature_matcher_factory, FeatureMatcherTypes
from tqdm import tqdm
from feature_tracker_configs import FeatureTrackerConfigs
%matplotlib inline

## Visual Odometry

In [9]:
folders = os.listdir('../data/dataset/sequences/')
folders.sort()
# folders.remove('03')
folders = folders[3:]
folders

['03', '04', '05', '06', '07', '08', '09']

In [10]:
exp_name = 'Intial_test'

In [12]:
for f in folders:
#     f = '03'
    print('Folder: ',f)
    config = Config(f)

    dataset = dataset_factory(config.dataset_settings)
    groundtruth = groundtruth_factory(config.dataset_settings)

    cam = PinholeCamera(config.cam_settings['Camera.width'], config.cam_settings['Camera.height'],
                        config.cam_settings['Camera.fx'], config.cam_settings['Camera.fy'],
                        config.cam_settings['Camera.cx'], config.cam_settings['Camera.cy'],
                        config.DistCoef, config.cam_settings['Camera.fps'])

    num_features=2000  # how many features do you want to detect and track?

    # select your tracker configuration (see the file feature_tracker_configs.py) 
    # LK_SHI_TOMASI, LK_FAST
    # SHI_TOMASI_ORB, FAST_ORB, ORB, BRISK, AKAZE, FAST_FREAK, SIFT, ROOT_SIFT, SURF, SUPERPOINT, FAST_TFEAT
    tracker_config = FeatureTrackerConfigs.LK_FAST
    tracker_config['num_features'] = num_features

    feature_tracker = feature_tracker_factory(**tracker_config)
    # create visual odometry object 
    vo = VisualOdometry(cam, groundtruth, feature_tracker)

    # todo: add the trajectory visualization
    traj_img_size = 800
    traj_img = np.zeros((traj_img_size, traj_img_size, 3), dtype=np.uint8)
    half_traj_img_size = int(0.5*traj_img_size)
    draw_scale = 1

    # second loop for iterating over all the frame
    result = []  
    for img_id in tqdm(range(dataset.max_frame_id)):
        img = dataset.getImage(img_id)
        if img is not None:
            vo.track(img, img_id)
            tmp = np.reshape(np.hstack((vo.cur_R, vo.cur_t)), 12)
            result.append(' '.join([str(i) for i in tmp]))

    # Save the results in the text files
    res_base_path = os.path.join('../data/results/', exp_name)
    res_folder_path = os.path.join(res_base_path, f+'.txt')
    os.makedirs(res_base_path, exist_ok=True)

    txt_file=open(res_folder_path, 'a') 
    txt_file.writelines("%s\n" % i for i in result) 
    txt_file.close() 
#     break

  0%|          | 2/801 [00:00<00:53, 15.05it/s]

Folder:  03
Processing KITTI Sequence of lenght:  801
using groundtruth:  kitti
 LkFeatureTracker: num levels on LK pyr optic flow:  3


  1%|          | 2/271 [00:00<00:21, 12.70it/s]

Folder:  04
Processing KITTI Sequence of lenght:  271
using groundtruth:  kitti
 LkFeatureTracker: num levels on LK pyr optic flow:  3


  0%|          | 2/2761 [00:00<03:15, 14.13it/s]

...

  0%|          | 2/4071 [00:00<04:24, 15.41it/s]

Folder:  08
Processing KITTI Sequence of lenght:  4071
using groundtruth:  kitti
 LkFeatureTracker: num levels on LK pyr optic flow:  3


  0%|          | 2/1591 [00:00<01:40, 15.77it/s]

Folder:  09
Processing KITTI Sequence of lenght:  1591
using groundtruth:  kitti
 LkFeatureTracker: num levels on LK pyr optic flow:  3


100%|██████████| 1591/1591 [03:56<00:00,  6.74it/s]
